### 1. Importing Libraries

In [1]:
import time as t
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
import pandas as pd

In [2]:
res = requests.get('https://www.kaggle.com/datasets?page=1')

In [3]:
res
# 200 -> response scraped successfully

<Response [200]>

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.kaggle.com')

In [5]:
month_dct = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

### 2. Scraping Data from Kaggle

In [14]:
values = []
for page in tqdm(range(1,501)):
    link = 'https://www.kaggle.com/datasets?page='+ str(page)
    driver.get(link)
    t.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    sp = soup.find('ul', class_ = 'km-list km-list--three-line')
    for data in sp.find_all('li'):
        try:
            title = data.find('div', class_ = 'sc-bczRLJ sc-fEOsli sc-kIKDeO hWgGcP gYKcfE lowKEm').text
        except:
            title = np.nan
        try:
            author_id = "https://www.kaggle.com"+data.find('a', class_ = 'sc-jdAMXn sc-bZnhIo bdwTMI hagtLI').get('href').strip()
        except:
            author_id = np.nan
        try:
            dataset_link = 'https://www.kaggle.com'+data.find('a').get('href')
        except:
            dataset_link = np.nan
        try:
            image_link = data.find('div', class_ = 'sc-dZrFBT kmXhRk').get('style').split('"')[1].split('?')[0]
        except:
            image_link = np.nan
        try:
            author_name = data.find_all('span', class_ = 'sc-llJcti sc-iIPllB sc-dsQDmV dRaCtU piKwW hlxqVX')[0].text.split('·')[0].strip()
        except:
            author_name = np.nan
        try:
            date_time = data.find_all('span', class_ = 'sc-llJcti sc-iIPllB sc-dsQDmV dRaCtU piKwW hlxqVX')[0].find('span').get('title')[:-31].strip()
            date = date_time.split(" ")[1:-1]
            date[0] = str(month_dct[date[0]])
            date =  '/'.join(date)
            day = date_time.split(" ")[0]
            time = date_time.split(" ")[-1]
        except:
            date_time = np.nan
            date = np.nan
            day = np.nan
            time = np.nan
        try:
            size = data.find_all('span', class_ = 'sc-llJcti sc-iIPllB sc-dsQDmV dRaCtU piKwW hlxqVX')[1].text.split("·")[2].strip()
        except:
            size = np.nan
        try:
            file_info = data.find_all('span', class_ = 'sc-llJcti sc-iIPllB sc-dsQDmV dRaCtU piKwW hlxqVX')[1].text.split("·")[1].strip()
            no_of_files = file_info.split(' ')[0]
            file_type = file_info.split('(')[-1][:-1]
        except:
            file_info = np.nan
            no_of_files = np.nan 
            file_type = np.nan
        try:
            upvotes = data.find('span', mode = 'default').text.strip()
        except:
            upvotes = np.nan
        try:
            medals = data.find('span', class_ = 'sc-llJcti sc-iIPllB dRaCtU piKwW').text.strip()
        except:
            medals = np.nan
        try:
            usability = data.find_all('span', class_ = 'sc-llJcti sc-iIPllB sc-dsQDmV dRaCtU piKwW hlxqVX')[1].find('span').text[9:].strip()
        except:
            usability = np.nan
        
        
        values.append([title,author_name, author_id, no_of_files, size, file_type, upvotes, medals, usability, date, day, time, dataset_link, image_link])
    t.sleep(2)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [52:11<00:00,  6.26s/it]


### 3. Converting into Dataframe

In [16]:
df = pd.DataFrame(values, columns=['Dataset_name', 'Author_name','Author_id', 'No_of_files','size', 'Type_of_file', 'Upvotes', 'Medals', 'Usability', 'Date', 'Day', 'Time', 'Dataset_link', 'Image_link'])
df.head()

,Dataset_name,Author_name,Author_id,No_of_files,size,Type_of_file,Upvotes,Medals,Usability,Date,Day,Time,Dataset_link,Image_link
0,Hotel Reservations Dataset,Ahsan Raza,https://www.kaggle.com/ahsan81,1,491 kB,CSV,315,Silver,10.0,1/04/2023,Wed,18:20:31,https://www.kaggle.com/datasets/ahsan81/hotel-...,https://storage.googleapis.com/kaggle-datasets...
1,MusicCaps,Google Research,NaN,1,812 kB,CSV,188,Silver,9.4,1/25/2023,Wed,14:55:48,https://www.kaggle.com/datasets/googleai/music...,https://storage.googleapis.com/kaggle-datasets...
2,Most Subscribed 1000 Youtube Channels,Mrityunjay Pathak,https://www.kaggle.com/themrityunjaypathak,1,29 kB,CSV,76,Bronze,10.0,1/21/2023,Sat,20:12:05,https://www.kaggle.com/datasets/themrityunjayp...,https://storage.googleapis.com/kaggle-datasets...
3,Olympics 124 years Dataset(till 2020),Nitish Sharma01,https://www.kaggle.com/nitishsharma01,3,5 MB,CSV,30,Bronze,10.0,2/01/2023,Wed,14:30:49,https://www.kaggle.com/datasets/nitishsharma01...,https://storage.googleapis.com/kaggle-datasets...
4,Medical Student Mental Health,The Devastator,https://www.kaggle.com/thedevastator,2,19 kB,CSV,37,Bronze,10.0,1/25/2023,Wed,06:30:14,https://www.kaggle.com/datasets/thedevastator/...,https://storage.googleapis.com/kaggle-datasets...


### 4. Exporting Raw Dataset

In [17]:
df.to_csv('kaggle-raw.csv')

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9971 entries, 0 to 9970
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Dataset_name  9971 non-null   object
 1   Author_name   9971 non-null   object
 2   Author_id     9619 non-null   object
 3   No_of_files   9971 non-null   object
 4   size          9478 non-null   object
 5   Type_of_file  9971 non-null   object
 6   Upvotes       9971 non-null   object
 7   Medals        6180 non-null   object
 8   Usability     9970 non-null   object
 9   Date          9971 non-null   object
 10  Day           9971 non-null   object
 11  Time          9971 non-null   object
 12  Dataset_link  9971 non-null   object
 13  Image_link    9971 non-null   object
dtypes: object(14)
memory usage: 1.1+ MB


### 4. Checking Null values and removing unnecessary columns

In [19]:
df.isnull().sum()

Dataset_name       0
Author_name        0
Author_id        352
No_of_files        0
size             493
Type_of_file       0
Upvotes            0
Medals          3791
Usability          1
Date               0
Day                0
Time               0
Dataset_link       0
Image_link         0
dtype: int64

In [20]:
df['Medals'] = df['Medals'].fillna('No Medal')
del df['Image_link']
df.isnull().sum()

Dataset_name      0
Author_name       0
Author_id       352
No_of_files       0
size            493
Type_of_file      0
Upvotes           0
Medals            0
Usability         1
Date              0
Day               0
Time              0
Dataset_link      0
dtype: int64

In [21]:
df = df.dropna()
df.isna().sum()

Dataset_name    0
Author_name     0
Author_id       0
No_of_files     0
size            0
Type_of_file    0
Upvotes         0
Medals          0
Usability       0
Date            0
Day             0
Time            0
Dataset_link    0
dtype: int64

### 5. Converting it into CSV

In [23]:
len(df)

9158

In [24]:
df.to_csv('kaggle-preprocessed.csv', index = False)